Those first functions are to use google colab if you want to connect to it to use the dataset and install wandb and login to it

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
%cd ../drive/My Drive/hw2ml

In [ ]:
!pip install wandb
!wandb login

Download the dataset from kaggle on google drive or on the local computer

In [ ]:
#insert the kaggle json file in the content directory before running this cell
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d ibrahimserouis99/one-piece-image-classifier

In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('one-piece-image-classifier.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

classes of the dataset

In [ ]:
classnames=[
"Ace",
"Akainu",
"Brook" ,
"Chopper",
"Crocodile",
"Franky",
"Jinbei",
"Kurohige",
"Law",
"Luffy",
"Mihawk",
"Nami",
"Rayleigh",
"Robin",
"Sanji",
"Shanks",
"Usopp",
"Zoro"]

classes={name:i for i, name in enumerate(classnames)}
fromIndexToClass={i:name for i, name in enumerate(classnames)}



create the annotation file to read the images and their labels after

In [ ]:
#read the data folder and create a file with the annotations
import os 

with open("./annotations.txt", "w") as f:
    for i,n in enumerate(classes):
        for file in os.listdir("./Data/Data/"+n):
            f.write("./Data/Data/"+n+"/"+file+" "+str(i)+"\n")
    print(n+" done")

define the dataset to be used in pytorch

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import random

#set the seeds to make everything reproducible
torch.manual_seed(0)
np.random.seed(0)

#dataset class
class OnePieceDataset(Dataset):
    items=[]
    def __init__(self,w,h):
        self.w=w
        self.h=h
        with open("./annotations.txt", "r") as f:
            for line in f.readlines():
                self.items.append(line.split(" "))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item=self.items[idx]

        #one hot encoding of the class of the item
        label=torch.zeros(len(classes))
        label[int(item[-1].strip())]=1
        
        #path of the image
        path=" ".join(item[0:-1])

        #load the image as pil image
        image=Image.open(path).convert('RGB')
        image = image.resize((self.w, self.h)) 

        #convert it to a tensor
        image=torch.tensor(np.array(image),dtype=torch.float32)
        
        return image, label ,path

create a custom neural network to be used called mlp

In [ ]:
import torch
import torch.nn as nn

class Mlp(nn.Module):
    def __init__(self,w,h):
        self.w=w
        self.h=h
        super(Mlp, self).__init__()
        self.fc1 = nn.Linear(self.w*self.h*3, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 18)

    def forward(self, x):
        x = self.fc1(x)
        x=nn.ReLU()(x)

        x = self.fc2(x)
        x=nn.ReLU()(x)

        x=self.fc3(x)
        x=nn.ReLU()(x)

        x=self.fc4(x)
        x=nn.ReLU()(x)

        x=self.fc5(x)
        x=nn.ReLU()(x)

        x=self.fc6(x)
        
        #do not need to use softmax or sigmoid because we use cross entropy loss and it does it for us
        return x
    


read the dataset and split it 

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torchvision
import wandb

w,h=200,200

#read the dataset
dataset=OnePieceDataset(w,h)

#split in train and test set
split=[int(0.8*len(dataset)),int(0.2*len(dataset))+1]
train,test = torch.utils.data.random_split(dataset,split)



training loop to train the model and see the performances
if you dont want to use wandb set the variable to false

In [5]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score


#device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device= torch.device("cpu")
print(device)

#declare parameters
percentageOfDataset=1
num_epochs=10
batch_size=32
w_and_b=True
nn_type="resnetFrom0"

if w_and_b:
    wandb.init(project='hw2ml', entity='bbooss97',name=nn_type)

#types where have to change the input in the train and test
typesToChange=["resnetFrom0","resnetPretrainedFineTuneFc","resnetPretrainedFineTuneAll","mobilenetPretrainedFineTuneAll"]

#dataloader
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True , drop_last=True)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True , drop_last=True)


#define the model
if nn_type=="mlp":
    model=Mlp(w,h)
elif nn_type=="cnn":
    model=Cnn(w,h)
elif nn_type=="resnetPretrainedFineTuneFc":
    model=torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
    model.fc=torch.nn.Linear(512,18)
    toFreeze=[j for i,j in model.named_parameters()][:-2]
    for i in toFreeze:
        i.requires_grad=False
elif nn_type=="resnetPretrainedFineTuneAll":
    model=torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
    model.fc=torch.nn.Linear(512,18)
elif nn_type=="resnetFrom0":
    model=torch.hub.load('pytorch/vision:v0.6.0', 'resnet18',pretrained=False)
    model.fc=torch.nn.Linear(512,18)
elif nn_type=="mobilenetPretrainedFineTuneAll":
    model=torchvision.models.mobilenet_v3_small()
    model.classifier[3]=torch.nn.Linear(1024,18)

if w_and_b:
    wandb.watch(model)

model.to(device)

#define loss and the optimizer
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):

    #train
    model.train()
    if w_and_b:
        wandb.log({"epoch":epoch})

    #train loop
    for i, (images, labels,paths) in enumerate(train_dataloader):

        #use just a percentage of the dataset
        if i>len(train_dataloader)*percentageOfDataset:
            break

        #move the data to the device
        images=images.to(device)
        labels=labels.to(device)

        #reshape the images
        images=images.reshape(batch_size,-1)

        #change the type of the input
        if nn_type in typesToChange:
            images =images.reshape(batch_size,w,h,3)
            images=torch.einsum("abcd->adbc",images)

        #forward pass
        outputs=model(images)
        
        #calculate the loss
        l=loss(outputs,labels)
        
        #backpropagation
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        #print the loss
        print("epoch: {}/{}, step: {}/{}, loss: {}".format(epoch+1,num_epochs,i+1,int(len(train_dataloader)*percentageOfDataset),l.item()))
        if w_and_b:
            if i%20==0:
                wandb.log({"loss_train":l.item()})
        
    #test
    model.eval()
    if w_and_b:
        wandb.watch(model)

    # Initialize variables to store metrics
    l = 0.0
    accuracy = 0.0
    f1 =0.0
    precision =0.0
    recall =0.0

    # Initialize lists to store predictions and labels
    testOutputs=[]
    testLabels=[]

    # Loop over the data in the test set
    with torch.no_grad():
        for i,(images, labels,paths) in enumerate(test_dataloader):

            # Move the data to the device
            images = images.to(device)
            labels = labels.to(device)

            # Reshape the images
            images=images.reshape(batch_size,-1)

            #change the input for those models
            if nn_type in typesToChange:
                images =images.reshape(batch_size,w,h,3)
                images=torch.einsum("abcd->adbc",images)

            # Forward pass: compute predictions and loss
            outputs = model(images)

            # Store predictions and labels
            testOutputs.append(outputs)
            testLabels.append(labels)

            ls = loss(outputs, labels)

            # Compute running metrics
            l += ls.item()
        

    # Concatenate predictions and labels
    testOutputs = torch.cat(testOutputs, dim=0)
    testLabels = torch.cat(testLabels, dim=0)

    testLabels=testLabels.cpu()
    testOutputs=testOutputs.cpu()

    # Compute average loss
    avg_loss = l / len(test_dataloader)
    
    # Compute accuracy
    accuracy = accuracy_score(testLabels.argmax(dim=1), testOutputs.argmax(dim=1))

    # Compute f1 score
    f1 = f1_score(testLabels.argmax(dim=1), testOutputs.argmax(dim=1), average='macro')

    # Compute precision
    precision = precision_score(testLabels.argmax(dim=1), testOutputs.argmax(dim=1), average='macro')

    # Compute recall
    recall = recall_score(testLabels.argmax(dim=1), testOutputs.argmax(dim=1), average='macro')

    # Print the metrics
    print(f'Test loss: {avg_loss:.4f}')
    print(f'Test accuracy: {accuracy:.4f}')
    print(f'Test f1: {f1:.4f}')
    print(f'Test precision: {precision:.4f}')
    print(f'Test recall: {recall:.4f}')

    if w_and_b:
        wandb.log({"avg_loss_test":avg_loss,"accuracy_test":accuracy,"f1_test":f1,"precision_test":precision,"recall_test":recall})

    #save the model
    torch.save(model,"./"+nn_type+".pt")

if w_and_b:
    wandb.finish()

print("finished")

function to visualize examples of the predictions fo the images with also the ground thruth

In [ ]:
import matplotlib.pyplot as plt

#use the models trained to predict some images from the test set
#types can be "mlp","resnetPretrainedFineTuneFc","resnetPretrainedFineTuneAll","resnetFrom0","mobilenetPretrainedFineTuneAll"
nn_type="resnetFrom0"

#load the model
model=torch.load("./"+nn_type+".pt")
model.eval()

#types where have to change the input in the train and test
typesToChange=["resnetFrom0","resnetPretrainedFineTuneFc","resnetPretrainedFineTuneAll","mobilenetPretrainedFineTuneAll"]

#load the images
images=[]
labels=[]
paths=[]

#how many images to predict
howManyImages=30

for i in range(howManyImages):
    index=random.randint(0,len(test)-1)
    images.append(test[index][0])
    labels.append(test[index][1])
    paths.append(test[index][2])

#stack the labels
labels =torch.stack(labels)

#stack the imagess and reshape
images =torch.stack(images)
images= images.reshape(howManyImages,-1)

#change the input for those models
if nn_type in typesToChange:
    images =images.reshape(howManyImages,w,h,3)
    images=torch.einsum("abcd->adbc",images)

#predict
outputs=model(images)
outputs=outputs.argmax(dim=1)

labels=labels.argmax(dim=1)

#show the images
for i in range(howManyImages):
    
    #show image from path
    img = Image.open(paths[i])
    plt.imshow(img)
    
    plt.title("predicted: "+str(fromIndexToClass[outputs[i].item()])+" real: "+str(fromIndexToClass[labels[i].item()]))
    plt.show()

print("finished")





